In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Switch directories
%cd /content/gdrive/MyDrive/Colab\ Data/COCO-Human-Pose
%ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose
coco_df.py         dataset.py           LICENSE     output/           scripts/
constants.py       evaluation.py        logs/       __pycache__/      test.py
data/              hourglass_blocks.py  models/     README.md         train.py
data_generator.py  hourglass.py         notebooks/  requirements.txt


In [12]:
from coco_df import get_df
import pandas as pd

df = get_df()

loading annotations into memory...
Done (t=10.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [11]:
# Write df to csv (images with only one person and at least 10 kps)
anns_per_img = df.groupby('src_set_image_id',as_index=False).size()
imgs_w_one_ann = anns_per_img.loc[anns_per_img['size'] == 1]
imgs_w_one_ann = pd.merge(imgs_w_one_ann,df)
one_ann_min10kp_imgs = imgs_w_one_ann.loc[imgs_w_one_ann['num_keypoints'] > 10]
one_ann_min10kp_imgs
csv_data = one_ann_min10kp_imgs.to_csv('data/one_ann_min10kp_imgs.csv',index=False)